In [1]:
import cv2
import numpy as np

a=['Predictive_Modeling_Model_Performance_1.jpg','Revenue_Dashboard_2.jpg','SalesForecasting_4.jpg','Retail_Sales_Distribution_Network_Dashboard_With_Forecasting_3(1).jpg']
image = cv2.imread('img/'+a[0])



# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
bilateral = cv2.bilateralFilter(gray, 5,40,40)
adaptive_threshold_bi = cv2.adaptiveThreshold(bilateral, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 3)
#cv2.imshow("adaptive_bilateral_filter", adaptive_threshold_bi)
# Apply Gaussian blur to reduce noise
# blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply edge detection
edges = cv2.Canny(adaptive_threshold_bi, 50, 150)


In [2]:
# Find contours in the edges image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through the contours and filter out small or irrelevant ones
min_area = 500  # Minimum area to filter small contours (you can adjust based on your image)
subgraph_contours = []

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > min_area:  # Only consider contours with area above the threshold
        subgraph_contours.append(cnt)
        # Draw the contours on the image to visualize
        cv2.drawContours(image, [cnt], -1, (0, 255, 0), 2)


In [13]:
# Approximate each contour to a polygonal shape
for cnt in subgraph_contours:
    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    
    # If the contour approximates a rectangle (4 vertices), it could be a subgraph
    if len(approx) == 4:
        x, y, w, h = cv2.boundingRect(approx)  # Get bounding box for the subgraph
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

cv2.imshow('Image with Rectangles', image)
cv2.waitKey(0)
cv2.destroyAllWindows()